In [75]:
from dbase.DataAPI.ThetaData import * #type: ignore
from dbase.database.SQLHelpers import * #type: ignore
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from EventDriven.eventScheduler import *
from trade.backtester_.backtester_ import PTDataset, PTBacktester
import pandas_ta as ta
from trade.assets.Stock import Stock
from trade.backtester_.utils.WalkForwardUtils import prev_monday 
from trade.backtester_.strats import MAStrat
import yfinance as yf
from datetime import datetime
from EventDriven.backtest import OptionSignalBacktest
%load_ext autoreload
%autoreload 2
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
import json
with open('../input/profitable_weights.json', 'r') as f:
    weights = json.load(f)
ttrades_ = pd.read_csv('../input/profitable_trades.csv').iloc[:, 1:]

w_map = {x: w  * 0.75 for x, w in weights.items()}
ttrades_

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
0,34.0,504.0,524.0,279.795877,253.010232,-910.711955,-0.095733,2023-07-05,2023-08-02,28 days,TSLA
1,28.0,504.0,526.0,192.240502,185.493749,-188.909083,-0.035095,2023-07-05,2023-08-04,30 days,AAPL
2,17.0,504.0,529.0,336.262811,322.011093,-242.279208,-0.042383,2023-07-05,2023-08-09,35 days,MSFT
3,48.0,504.0,536.0,87.043588,82.000000,-242.092217,-0.057943,2023-07-05,2023-08-18,44 days,AVGO
4,11.0,504.0,583.0,130.695846,122.257034,-92.826927,-0.064568,2023-07-05,2023-10-25,112 days,AMZN
5,178.0,504.0,753.0,42.282471,123.470001,14451.380412,1.920123,2023-07-05,2024-07-01,362 days,NVDA
6,1.0,522.0,522.0,170.283917,168.355807,-1.928110,-0.011323,2023-07-31,2023-07-31,0 days,JNJ
7,5.0,522.0,556.0,332.810769,318.700396,-70.551863,-0.042398,2023-07-31,2023-09-18,49 days,HD
8,5.0,530.0,535.0,239.284572,224.549052,-73.677602,-0.061582,2023-08-10,2023-08-17,7 days,BA
9,5.0,530.0,535.0,53.784257,52.084613,-8.498219,-0.031601,2023-08-10,2023-08-17,7 days,WMT


In [63]:
new_trades = ttrades_.copy()[:10]

In [64]:
trades_ = ttrades_.copy()[:5]
trades_

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
0,34.0,504.0,524.0,279.795877,253.010232,-910.711955,-0.095733,2023-07-05,2023-08-02,28 days,TSLA
1,28.0,504.0,526.0,192.240502,185.493749,-188.909083,-0.035095,2023-07-05,2023-08-04,30 days,AAPL
2,17.0,504.0,529.0,336.262811,322.011093,-242.279208,-0.042383,2023-07-05,2023-08-09,35 days,MSFT
3,48.0,504.0,536.0,87.043588,82.000000,-242.092217,-0.057943,2023-07-05,2023-08-18,44 days,AVGO
4,11.0,504.0,583.0,130.695846,122.257034,-92.826927,-0.064568,2023-07-05,2023-10-25,112 days,AMZN


In [ ]:
#Backtest class 
evb_backtest = OptionSignalBacktest(ttrades_)

In [78]:
evb_backtest.portfolio.weight_map = w_map
evb_backtest.portfolio.weight_map
evb_backtest.portfolio.order_settings = {'type': 'naked',
 'specifics': [{'direction': 'long',
   'rel_strike': .900,
   'dte': 365,
   'moneyness_width': 0.1},
   {'direction': 'short',
   'rel_strike': .80,
   'dte': 365,
   'moneyness_width': 0.1}],

 'name': 'vertical_spread'}

evb_backtest.portfolio.order_settings

The following symbols: ['NVDA', 'QCOM', 'MU', 'GOOG', 'AMD', 'BAC', 'BA', 'SBUX', 'DIS', 'HD', 'WMT', 'PFE', 'JNJ', 'INTC'] are not being processed but present in weight_map


{'type': 'naked',
 'specifics': [{'direction': 'long',
   'rel_strike': 0.9,
   'dte': 365,
   'moneyness_width': 0.1},
  {'direction': 'short',
   'rel_strike': 0.8,
   'dte': 365,
   'moneyness_width': 0.1}],
 'name': 'vertical_spread'}

In [67]:
signals = evb_backtest.bars.signal_df
# evb_backtest.bars.signal_df['TSLA'] = 0
# evb_backtest.bars.signal_df['AMZN'] = 0
# evb_backtest.bars.signal_df['AVGO'] = 0
# evb_backtest.bars.signal_df['AAPL'] = 0
# evb_backtest.bars.signal_df.loc[0, 'MSFT'] = 1
evb_backtest.bars.signal_df.loc[35, 'MSFT'] = 0
evb_backtest.bars.signal_df.loc[61, 'MSFT'] = -1
# evb_backtest.bars.signal_df.loc[63, 'MSFT'] = -1
evb_backtest.bars.signal_df
# evb_backtest.portfolio.max_contract_price['MSFT'] = 0.1

,Date,TSLA,AAPL,MSFT,AVGO,AMZN
0,2023-07-05,1,1,1,1,1
1,2023-07-06,0,0,0,0,0
2,2023-07-07,0,0,0,0,0
3,2023-07-10,0,0,0,0,0
4,2023-07-11,0,0,0,0,0
5,2023-07-12,0,0,0,0,0
6,2023-07-13,0,0,0,0,0
7,2023-07-14,0,0,0,0,0
8,2023-07-17,0,0,0,0,0
9,2023-07-18,0,0,0,0,0


In [79]:
pd.DataFrame(evb_backtest.events.events_dict).set_index('datetime').sort_index()

,type,symbol,signal_type,signal_id,order_settings
datetime,,,,,
2023-07-05,MARKET,NaN,NaN,NaN,NaN
2023-07-05,SIGNAL,AAPL,LONG,AAPL20230705LONG,NaN
2023-07-05,SIGNAL,MSFT,LONG,MSFT20230705LONG,NaN
2023-07-05,SIGNAL,AVGO,LONG,AVGO20230705LONG,NaN
2023-07-05,SIGNAL,AMZN,LONG,AMZN20230705LONG,NaN
2023-07-05,SIGNAL,TSLA,LONG,TSLA20230705LONG,NaN
2023-07-06,MARKET,NaN,NaN,NaN,NaN
2023-07-07,MARKET,NaN,NaN,NaN,NaN
2023-07-10,MARKET,NaN,NaN,NaN,NaN


In [69]:
import cProfile
import pstats
import io

profiler = cProfile.Profile()
profiler.enable()
#run backtest  

evb_backtest.run()
profiler.disable()
stream = io.StringIO()
stats = pstats.Stats(profiler, stream=stream).sort_stats('cumulative')

Processing event: MARKET
Processing event: SIGNAL
Processing event: SIGNAL
Processing event: SIGNAL
Processing event: SIGNAL
Not generating order because:TOO_ILLIQUID SignalEvent type:LONG, symbol=AVGO, date:2023-07-05 00:00:00, Order Settings=None, signal_id:AVGO20230705LONG
Processing event: SIGNAL
Processing event: ORDER
Processing event: ORDER
Processing event: ORDER
Processing event: ORDER
Processing event: FILL
Processing event: FILL
Processing event: FILL
Processing event: FILL
Event queue is empty, processed 14 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1

In [71]:
evb_backtest.portfolio.trades

,Ticker,PnL,ReturnPct,EntryPrice,EntryCommission,EntrySlippage,EntryMarketValue,TotalEntryCost,AuxilaryEntryCost,ExitPrice,ExitCommission,ExitSlippage,ExitMarketValue,TotalExitCost,AuxilaryExitCost,Quantity,EntryTime,ExitTime,Duration,Positions,SignalID
0,TSLA,-19.184526,-0.005600,3425.778361,1.116,5.440721,6850.440721,68.515567,6.556721,3416.186098,1.116,-11.511804,6833.488196,68.323722,12.627804,2,2023-07-05,2023-08-02,28,&L:TSLA20240621C283.33&S:TSLA20240621C400,TSLA20230705LONG
1,AAPL,-11.415639,-0.006104,1870.116929,1.116,6.117859,3739.117859,37.402339,7.233859,1864.409110,1.116,-3.065780,3729.934220,37.288182,4.181780,2,2023-07-05,2023-08-04,30,&L:AAPL20240621C195&S:AAPL20240621C270,AAPL20230705LONG
2,MSFT,-9.680356,-0.003084,3138.863680,0.558,5.805680,3138.305680,31.388637,6.363680,3129.183324,0.558,-2.758676,3129.741324,31.291833,3.316676,1,2023-07-05,2023-08-09,35,&L:MSFT20240621C345&S:MSFT20240621C450,MSFT20230705LONG
3,AMZN,-3.950009,-0.002587,1526.671795,0.558,1.113795,1526.113795,15.266718,1.671795,1522.721785,0.558,-1.720215,1523.279785,15.227218,2.278215,1,2023-07-05,2023-10-25,112,&L:AMZN20240621C132.5&S:AMZN20240621C185,AMZN20230705LONG


In [74]:
events_df = evb_backtest.events.events
events_df = events_df[events_df['signal_id'] == 'AMZN20230705LONG']
events_df

,type,symbol,signal_type,signal_id,order_settings,order_type,quantity,direction,position,exchange,fill_cost,market_value,slippage,commission
datetime,,,,,,,,,,,,,,
2023-07-05,SIGNAL,AMZN,LONG,AMZN20230705LONG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-05,ORDER,AMZN,NaN,AMZN20230705LONG,NaN,MKT,1.0,BUY,"{'long': ['AMZN20240621C132.5'], 'short': ['AM...",NaN,NaN,NaN,NaN,NaN
2023-07-05,FILL,AMZN,NaN,AMZN20230705LONG,NaN,NaN,1.0,BUY,"{'long': ['AMZN20240621C132.5'], 'short': ['AM...",ARCA,15.266718,15.261138,0.011138,0.00558
2023-10-25,ORDER,AMZN,NaN,AMZN20230705LONG,NaN,MKT,1.0,SELL,"{'long': ['AMZN20240621C132.5'], 'short': ['AM...",NaN,NaN,NaN,NaN,NaN
2023-10-25,FILL,AMZN,NaN,AMZN20230705LONG,NaN,NaN,1.0,SELL,"{'long': ['AMZN20240621C132.5'], 'short': ['AM...",ARCA,15.227218,15.232798,-0.017202,0.00558


In [11]:
evb_backtest.clean_run(trades=new_trades)

Processing event: MARKET
Processing event: SIGNAL
Processing event: SIGNAL
Processing event: SIGNAL
Processing event: SIGNAL
Not generating order because:TOO_ILLIQUID SignalEvent type:LONG, symbol=AVGO, date:2023-07-05 00:00:00, Order Settings=None, signal_id:AVGO20230705LONG
Processing event: SIGNAL
Processing event: SIGNAL
Processing event: ORDER
Processing event: ORDER
Processing event: ORDER
Processing event: ORDER
Processing event: ORDER
Processing event: FILL
Processing event: FILL
Processing event: FILL
Processing event: FILL
Processing event: FILL
Event queue is empty, processed 17 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, process

KeyboardInterrupt: 

In [12]:
evb_backtest.portfolio.trades_dict

{'&L:TSLA20240621C260&S:TSLA20240621C370': {'entry_price': 3926.3755205046727,
  'entry_date': Timestamp('2023-07-05 00:00:00'),
  'quantity': 2,
  'symbol': 'TSLA',
  'entry_commission': 1.1159999999999999,
  'entry_market_value': 7851.6350410093455,
  'entry_slippage': 6.6350410093434675,
  'total_entry_cost': 78.52751041009346,
  'auxilary_entry_cost': 7.751041009343467,
  'signal_id': 'TSLA20230705LONG',
  'trade_id': '&L:TSLA20240621C260&S:TSLA20240621C370',
  'exit_price': 3914.302534382505,
  'exit_date': Timestamp('2023-08-02 00:00:00'),
  'exit_commission': 1.1159999999999999,
  'exit_slippage': -15.278931234990978,
  'exit_market_value': 7829.72106876501,
  'total_exit_cost': 78.2860506876501,
  'auxilary_exit_cost': 16.394931234990977,
  'pnl': -24.1459722443351,
  'return_pct': -0.0061496848985120815,
  'duration_days': 28,
  'exit_method': 'sell'},
 '&L:AAPL20240621C175&S:AAPL20240621C255': {'entry_price': 3058.041873283369,
  'entry_date': Timestamp('2023-07-05 00:00:00')

In [51]:
tsla = evb_backtest.portfolio.trades.loc['&L:TSLA20240621C260&S:TSLA20240621C370']
tsla['entry_date']

Timestamp('2023-07-05 00:00:00')

In [ ]:
(pd.to_datetime('2024-06-21') - pd.to_datetime('2024-06-21')).days

0

In [57]:
tsla_option = evb_backtest.portfolio.get_option_data('TSLA20240621C260')
tsla_option = tsla_option[~tsla_option.index.duplicated(keep='last')]
tsla_option = tsla_option.loc[tsla['entry_date']]
tsla_option

Open                  73.530000
High                  74.000000
Low                   71.200000
Close                 74.000000
Volume                78.000000
Bid_size             183.000000
CloseBid              72.650000
Ask_size             396.000000
CloseAsk              73.950000
Midpoint              73.300000
Weighted_midpoint     73.539119
Name: 2023-07-05 00:00:00, dtype: float64

In [41]:
evb_backtest.portfolio.get_all_positions()

long               short  \
datetime   symbol                                           
2023-07-05 TSLA    [TSLA20240621C260]  [TSLA20240621C370]   
           AAPL    [AAPL20240621C175]  [AAPL20240621C255]   
           MSFT    [MSFT20240621C315]  [MSFT20240621C450]   
           AMZN    [AMZN20240621C120]  [AMZN20240621C170]   
           NVDA    [NVDA20240621C410]  [NVDA20240621C550]   
2023-07-06 TSLA    [TSLA20240621C260]  [TSLA20240621C370]   
           AAPL    [AAPL20240621C175]  [AAPL20240621C255]   
           MSFT    [MSFT20240621C315]  [MSFT20240621C450]   
           AMZN    [AMZN20240621C120]  [AMZN20240621C170]   
           NVDA    [NVDA20240621C410]  [NVDA20240621C550]   
2023-07-07 TSLA    [TSLA20240621C260]  [TSLA20240621C370]   
           AAPL    [AAPL20240621C175]  [AAPL20240621C255]   
           MSFT    [MSFT20240621C315]  [MSFT20240621C450]   
           AMZN    [AMZN20240621C120]  [AMZN20240621C170]   
           NVDA    [NVDA20240621C410]  [NVDA20240621C550]   
2023-07-10 TSLA    [TSLA20240621C260]  [TSLA20240621C370]   
           AAPL    [AAPL20240621C175]  [AAPL20240621C255]   
           MSFT    [MSFT20240621C315]  [MSFT20240621C450]   
           AMZN    [AMZN20240621C120]  [AMZN20240621C170]   
           NVDA    [NVDA20240621C410]  [NVDA20240621C550]   
2023-07-11 TSLA    [TSLA20240621C260]  [TSLA20240621C370]   
           AAPL    [AAPL20240621C175]  [AAPL20240621C255]   
           MSFT    [MSFT20240621C315]  [MSFT20240621C450]   
           AMZN    [AMZN20240621C120]  [AMZN20240621C170]   
           NVDA    [NVDA20240621C410]  [NVDA20240621C550]   
2023-07-12 TSLA    [TSLA20240621C260]  [TSLA20240621C370]   
           AAPL    [AAPL20240621C175]  [AAPL20240621C255]   
           MSFT    [MSFT20240621C315]  [MSFT20240621C450]   
           AMZN    [AMZN20240621C120]  [AMZN20240621C170]   
           NVDA    [NVDA20240621C410]  [NVDA20240621C550]   
2023-07-13 TSLA    [TSLA20240621C260]  [TSLA20240621C370]   
           AAPL    [AAPL20240621C175]  [AAPL20240621C255]   
           MSFT    [MSFT20240621C315]  [MSFT20240621C450]   
           AMZN    [AMZN20240621C120]  [AMZN20240621C170]   
           NVDA    [NVDA20240621C410]  [NVDA20240621C550]   
2023-07-14 TSLA    [TSLA20240621C260]  [TSLA20240621C370]   
           AAPL    [AAPL20240621C175]  [AAPL20240621C255]   
           MSFT    [MSFT20240621C315]  [MSFT20240621C450]   
           AMZN    [AMZN20240621C120]  [AMZN20240621C170]   
           NVDA    [NVDA20240621C410]  [NVDA20240621C550]   
2023-07-17 TSLA    [TSLA20240621C260]  [TSLA20240621C370]   
           AAPL    [AAPL20240621C175]  [AAPL20240621C255]   
           MSFT    [MSFT20240621C315]  [MSFT20240621C450]   
           AMZN    [AMZN20240621C120]  [AMZN20240621C170]   
           NVDA    [NVDA20240621C410]  [NVDA20240621C550]   
2023-07-18 TSLA    [TSLA20240621C260]  [TSLA20240621C370]   
           AAPL    [AAPL20240621C175]  [AAPL20240621C255]   
           MSFT    [MSFT20240621C315]  [MSFT20240621C450]   
           AMZN    [AMZN20240621C120]  [AMZN20240621C170]   
           NVDA    [NVDA20240621C410]  [NVDA20240621C550]   
2023-07-19 TSLA    [TSLA20240621C260]  [TSLA20240621C370]   
           AAPL    [AAPL20240621C175]  [AAPL20240621C255]   
           MSFT    [MSFT20240621C315]  [MSFT20240621C450]   
           AMZN    [AMZN20240621C120]  [AMZN20240621C170]   
           NVDA    [NVDA20240621C410]  [NVDA20240621C550]   
2023-07-20 TSLA    [TSLA20240621C260]  [TSLA20240621C370]   
           AAPL    [AAPL20240621C175]  [AAPL20240621C255]   
           MSFT    [MSFT20240621C315]  [MSFT20240621C450]   
           AMZN    [AMZN20240621C120]  [AMZN20240621C170]   
           NVDA    [NVDA20240621C410]  [NVDA20240621C550]   
2023-07-21 TSLA    [TSLA20240621C260]  [TSLA20240621C370]   
           AAPL    [AAPL20240621C175]  [AAPL20240621C255]   
           MSFT    [MSFT20240621C315]  [MSFT20240621C450]   
           AMZN    [AMZN20240621C120]  [AMZN20240621C170]   
       

In [ ]:
import re
input_string = "&L:AMZN20240621C120&S:AMZN20240621C170"

# Regular expression to match the desired patterns
pattern = r'&[LS]:(\w+)'

# Find all matches
matches = re.findall(pattern, input_string)

# Output the results
for match in matches:
    print(match)

AMZN20240621C120
AMZN20240621C170
BA20240621C220
BA20240621C310


In [32]:
from pandas.tseries.holiday import USFederalHolidayCalendar
HOLIDAY_SET = set(USFederalHolidayCalendar().holidays(start='2022-08-16', end='2022-08-16').strftime('%Y-%m-%d'))
HOLIDAY_SET

set()

In [15]:
order_result = evb_backtest.risk_manager.OrderPicker.get_order('MSFT', '2023-09-05', 'C', 5.0,  {'type': 'naked',
 'specifics': [{'direction': 'long',
   'rel_strike': .900,
   'dte': 365,
   'moneyness_width': 0.02},
   {'direction': 'short',
   'rel_strike': .80,
   'dte': 365,
   'moneyness_width': 0.02}],

 'name': 'vertical_spread'})

print(order_result)

{'result': 'TOO_ILLIQUID', 'data': None}


In [84]:
stats.print_stats(30)
print(stream.getvalue())
with open('bactest_data.txt', 'w') as f:
  stream.seek(0)
  f.write(stream.read())
  f.flush()

         179946728 function calls (179230715 primitive calls) in 771.171 seconds

   Ordered by: cumulative time
   List reduced from 2996 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000  771.173  385.587 c:\Users\Zino\python-playground\QuantTools\.venv\lib\site-packages\IPython\core\interactiveshell.py:3517(run_code)
        2    0.000    0.000  771.173  385.587 {built-in method builtins.exec}
        1    0.000    0.000  771.173  771.173 C:\Users\Zino\AppData\Local\Temp\ipykernel_28000\3226836942.py:1(<module>)
        1    0.028    0.028  771.173  771.173 C:\Users\Zino\python-playground\QuantTools\EventDriven\backtest.py:37(run)
       79    0.000    0.000  629.085    7.963 C:\Users\Zino\python-playground\QuantTools\EventDriven\portfolio.py:378(update_signal)
       79    0.003    0.000  629.083    7.963 C:\Users\Zino\python-playground\QuantTools\EventDriven\portfolio.py:267(generate_order)
       45 

In [31]:
evb_backtest.portfolio.plot_portfolio()

AttributeError: 'OptionSignalPortfolio' object has no attribute '_trades'

In [ ]:
trades_df = evb_backtest.portfolio.get_trades().head(5)
trades_df

,Ticker,PnL,ReturnPct,EntryPrice,EntryCommission,EntrySlippage,EntryMarketValue,TotalEntryCost,AuxilaryEntryCost,ExitPrice,ExitCommission,ExitSlippage,ExitMarketValue,TotalExitCost,AuxilaryExitCost,Quantity,EntryTime,ExitTime,Duration,Positions
0,TSLA,-10806.520059,-21.677784,498.506687,6.417,16.736810,11459.236810,11488.807620,-29.570810,390.441487,6.417,16.571196,8986.571196,9003.142393,-16.571196,23,2023-07-05,2023-08-02,28,&L:TSLA20240621C330&S:TSLA20240621C346.67
1,AAPL,3656.985131,7.479087,488.961456,3.627,-10.628066,6352.871934,6349.497867,3.374066,525.531308,3.627,-2.465999,6835.534001,6833.068001,2.465999,13,2023-07-05,2023-08-04,30,&L:AAPL20240621C230&S:AAPL20240621C280
2,MSFT,-10309.332567,-21.247498,485.202187,3.906,-1.075379,6788.924621,6795.661241,-6.736621,382.108862,3.906,-1.569939,5353.430061,5351.860122,1.569939,14,2023-07-05,2023-08-09,35,&L:MSFT20240621C355&S:MSFT20240621C365
3,AMZN,-5391.261254,-10.873383,495.821865,0.837,1.628594,1486.628594,1489.931189,-3.302594,441.909252,0.837,-0.935243,1326.564757,1325.629513,0.935243,3,2023-07-05,2023-10-25,112,&L:AMZN20240621C145&S:AMZN20240621C160


In [ ]:
evb_backtest.portfolio.get_weighted_holdings()s

In [ ]:

pd.set_option('display.max_rows', 10000)
evb_backtest.portfolio.get_all_positions()

long                  short  \
datetime   symbol                                              
2023-07-05 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-06 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-07 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-10 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-11 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-12 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-13 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-14 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-17 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-18 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-19 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-20 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-21 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
 

: 